In [1]:
import pandas as pd
import requests
import json
import numpy as np
import time

# Setting  up

In [2]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('9wXh5oa4cfW07_eUn5Hu3A', 'm3GnhaEvbM5LGCWX3BMghLCatOLN3g')
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'babi_research/0.0.1'}
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'babi_research/0.0.1',
        'refresh_token': '167791397396-1cjKrPIOvr4ct1EFwMm57hY1vi7baw'
        }

In [3]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-2mkJgd08DRCF-4kpV8Q_2icxI9uPPg',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [4]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [ ]:
#Get moderators of feminisms
fds_mod= requests.get('https://oauth.reddit.com/r/femaledatingstrategy/about/moderators', headers=headers).json()

In [ ]:
fds_mod_df = pd.DataFrame(fds_mod["data"]["children"])
fds_mod_df

In [ ]:
fds_mod_df.iloc[6,]

# Function

In [7]:
def clean_data(source_dir, comment_bool):
    df = pd.read_csv(source_dir)
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [8]:
def clean_df(df, comment_bool):
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [19]:
def get_comments(df, start_from) :
    tic = time.time()
    global res_df
    res_df = []
    i = 0
    if start_from == None:
        N = 0
    else:
        N = start_from
    while i < len(df["id"]):
        j = min(len(df["id"]), i + 100)
        x = ",".join(df["id"][i:j])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()
        try:
            for k in range(j-i):
                comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
                res_df.append(comment_df)
        except IndexError:
            for k in range(len(comments.get("data").get("children"))):
                comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
                res_df.append(comment_df)
        i += 100
        if i%150000 == 0:
            res_out = pd.concat(res_df)
            res_out.to_csv('unpop_comments_' + str((i+start_from)/150000) + '.csv', encoding = 'utf_8_sig')
            res = []
        if i%300 == 0:
            time.sleep(2)
            #now = time.localtime(time.time())
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
            #print(time.strftime("Time now: %HH:%M:%S",now),f', i = {i}')
    return pd.concat(res_df)

In [ ]:
def get_submissions(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        submissions = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x}).json()
        for k in range(20):
            submission_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(submission_df)
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df  

# Scraping

# r/femaledatingstrategy

In [ ]:
fds = clean_data("E:/gihub-data/redditbots/fds/fds_comments.csv", comment_bool= True)

In [ ]:
fds_res = get_comments(fds)

In [ ]:
fds_res.to_csv('fds_comment_retrieved.csv')

In [ ]:
requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': 't3_cipdyg'}).json()

In [ ]:
requests.get('https://oauth.reddit.com/user/')

## r/AskReddit

In [ ]:
askreddit = clean_data("E:/gihub-data/redditbots/control-fds/askreddit.csv", comment_bool= True)

In [ ]:
askreddit_res = get_comments(askreddit)

In [ ]:
askreddit_res.to_csv('E:/gihub-data/redditbots/control-fds/askreddit_comment_retrieved.csv')

# PurplePillDebate

In [ ]:
purplepill = clean_data("E:/gihub-data/redditbots/control-fds/purple.csv", comment_bool= True)

In [ ]:
purple_res = get_comments(purplepill)

In [ ]:
purple_res.to_csv('E:/gihub-data/redditbots/control-fds/purple_comment_retrieved.csv')

# r/AmItheAsshole

undone

In [ ]:
AITA = clean_data("E:/gihub-data/redditbots/control-fds/AITA.csv", comment_bool= True)

In [ ]:
AITA_res = get_comments(AITA)

# r/unpopularopinion

undone

In [5]:
unpop_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/unpopularopinion.csv",  skiprows= list(range(1,150600)))

In [20]:
unpop_1 = clean_df(unpop_df, comment_bool=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_26028\332450768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] ='t1_' + df['id'].astype(str)
C:\Users\nguye\AppData\Local\Temp\ipykernel_26028\332450768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))


In [21]:
unpop_res = get_comments(unpop_1, start_from = 464700) #start_from 150600

Time elapsed:  14  secs, i =  300
Time elapsed:  29  secs, i =  600
Time elapsed:  44  secs, i =  900
Time elapsed:  58  secs, i =  1200
Time elapsed:  73  secs, i =  1500
Time elapsed:  87  secs, i =  1800
Time elapsed:  101  secs, i =  2100
Time elapsed:  116  secs, i =  2400
Time elapsed:  130  secs, i =  2700
Time elapsed:  145  secs, i =  3000
Time elapsed:  159  secs, i =  3300
Time elapsed:  174  secs, i =  3600
Time elapsed:  188  secs, i =  3900
Time elapsed:  204  secs, i =  4200
Time elapsed:  219  secs, i =  4500
Time elapsed:  234  secs, i =  4800
Time elapsed:  248  secs, i =  5100
Time elapsed:  263  secs, i =  5400
Time elapsed:  278  secs, i =  5700
Time elapsed:  292  secs, i =  6000
Time elapsed:  307  secs, i =  6300
Time elapsed:  321  secs, i =  6600
Time elapsed:  336  secs, i =  6900
Time elapsed:  351  secs, i =  7200
Time elapsed:  367  secs, i =  7500
Time elapsed:  381  secs, i =  7800
Time elapsed:  396  secs, i =  8100
Time elapsed:  411  secs, i =  8400
T

KeyboardInterrupt: 

In [ ]:
unpop_res.to_csv('E:/gihub-data/redditbots/control-fds/unpop_comment_retrieved.csv')

In [10]:
unpop_df_test = unpop_df.iloc[0:10]

In [12]:
unpop_df_test['id'] ='t1_' + unpop_df_test['id'].astype(str)

C:\Users\nguye\AppData\Local\Temp\ipykernel_26028\1211801014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unpop_df_test['id'] ='t1_' + unpop_df_test['id'].astype(str)


In [13]:
x = ",".join(unpop_df_test["id"])

In [14]:
comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()

In [18]:
len(comments.get("data").get("children"))

10